# AA228/CS238 Optional Final Project: Escape Roomba

This notebook tests the QMDP + Branch & Bound implementation

In [1]:
# activate project environment
# include these lines of code in any future scripts/notebooks
#---
import Pkg
if !haskey(Pkg.installed(), "AA228FinalProject")
    jenv = joinpath(dirname(@__FILE__()), ".") # this assumes the notebook is in the same dir
    # as the Project.toml file, which should be in top level dir of the project. 
    # Change accordingly if this is not the case.
    Pkg.activate(jenv)
end
#---

"/home/zhangvwk/Documents/Stanford/aa228/final_project/AA228FinalProject/Project.toml"

In [2]:
# import necessary packages
using AA228FinalProject
using POMDPs
using POMDPPolicies
using BeliefUpdaters
using ParticleFilters
using POMDPSimulators
using Cairo
using Gtk
using Random
using Printf

┌ Info: Recompiling stale cache file /home/zhangvwk/.julia/compiled/v1.0/AA228FinalProject/uFJfC.ji for AA228FinalProject [fe2df5ea-4d44-4e5a-a895-9dbc87b19b37]
└ @ Base loading.jl:1190


## Define the POMDP

### Create state space, action space, sensor and construct POMDP

The QMDP offline method compute 1 alpha vector $\alpha_a$ per action, with components $\alpha_a(s)$ for $s \in \mathcal{S}, a \in \mathcal{A}$

This methods only works with finite state and action spaces, we first define the discretization/

Then we instantiate a Bump sensor. The Bumper indicates when contact has been made between any part of the Roomba and any wall.

Next, we instantiate the MDP, which defines the underlying simulation environment, assuming full observability. The MDP takes many arguments to specify details of the problem. One argument we must specify here is the ```config```. This argument, which can take values 1,2, or 3, specifies the room configuration, with each configuration corresponding to a different location for the goal and stairs.

Finally, we instantiate the POMDP. The POMDP takes as an argument the underlying MDP as well as the sensor, which it uses to define the observation model. 

In [3]:
# discrete state space
num_x_pts = 20
num_y_pts = 20
num_th_pts = 5
sspace = DiscreteRoombaStateSpace(num_x_pts,num_y_pts,num_th_pts);

In [4]:
# discrete action space
vlist = [0, 5, 10]
omlist = [-1, 0, 1]
aspace = vec(collect(RoombaAct(v, om) for v in vlist, om in omlist));

In [5]:
sensor = Bumper()
config = 1 # 1,2, or 3
m = RoombaPOMDP(sensor=sensor, mdp=RoombaMDP(config=config, sspace=sspace, aspace=aspace));
println("Number of discrete states:", n_states(m))
println("Number of discrete actions:", n_actions(m))

Number of discrete states:6000
Number of discrete actions:9


### Setting up a Particle Filter

Here, as the state space is high dimensional, we instantiate a particle filter.

First, we instantiate a resampler, which is responsible for updating the belief state given an observation. The first argument for both resamplers is the number of particles that represent the belief state. The lidar resampler takes a low-variance resampler as an additional argument, which is responsible for efficiently resampling a weighted set of particles. 

Next, we instantiate a ```SimpleParticleFilter```, which enables us to perform our belief updates.

Finally, we pass this particle filter into a custom struct called a ```RoombaParticleFilter```, which takes two additional arguments. These arguments specify the noise in the velocity and turn-rate, used when propegating particles according to the action taken. These can be tuned depending on the type of sensor used.

In [6]:
num_particles = 1000
resampler = BumperResampler(num_particles)

spf = SimpleParticleFilter(m, resampler)

v_noise_coefficient = 2.0
om_noise_coefficient = 0.5

belief_updater = RoombaParticleFilter(spf, v_noise_coefficient, om_noise_coefficient);

## Solve the POMDP

### Define a policy : Monte Carlo Search Tree

First we create a struct that subtypes the Policy abstract type, defined in the package ```POMDPPolicies.jl```. Here, we can also define certain parameters, such as a variable defining the depth 'd'.

Next, we define a function that can take in our policy and the belief state and return the desired action. We do this by defining a new ```POMDPs.action``` function that will work with our policy. 

In [40]:
using DataStructures # for DefaultDict

# Define the policy to test
mutable struct ToEnd1 <: Policy
    d::Int64 # depth 
    Q::DefaultDict{Any,Any,Int64}
    N::DefaultDict{Any,Any,Int64}
    T::Array{Any,1}
end
 
function pi_0(b::ParticleCollection{RoombaState}, aspace)
    return aspace[rand(1:length(aspace))] # Select an action randomly 
end

function Rollout(b::ParticleCollection{RoombaState}, d::Int64)
    n_part = n_particles(b)
    if d == 0
        return 0
    end
    # NEED TO DEFINE ROLLOUT POLICY PI_0
    a = pi_0(b, actions(m))
    j = rand(1:n_part)
    s = particle(b, j)
    sp, o, r = generate_sor(belief_updater.spf.model, s, a, belief_updater.spf.rng)
    bp = update(belief_updater, b, a, o)
    return r + discount(m)*Rollout(bp, d-1)
end

function Simulate(b::ParticleCollection{RoombaState}, h, d::Int64, Q, N, T)
    n_part = n_particles(b)
    c = 5.0 # exploration constant
    if d == 0
        return 0
    end

    if !(h in T)
        for a in actions(m)
            # Initialize N and Q values
            (N[(h,a)], Q[(h,a)]) = (0, 0)
        end
        T = vcat(T,h) # append h to search tree T
        return Rollout(b, d)
    end
    N[h] = sum([N[(h,a)] for a in actions(m)])
    a_ = findmax([Q[(h,a)] + c*sqrt(log(N[h])/N[(h,a)]) for a in actions(m)])[2]
    j = rand(1:n_part)
    s = particle(b, j)    
    sp, o, r = generate_sor(belief_updater.spf.model, s, a_, belief_updater.spf.rng)
    bp = update(belief_updater, b, a, o) 
    q = r + discount(m)*Simulate(bp, string(h,a_,o), d-1, Q, N, T)
    N[(h,a_)] += 1
    Q[(h,a_)] += (q - Q[(h,a_)])/N[(h,a_)]
    return Q
end
                          
# define a new function that takes in the policy struct and current belief,
# and returns the desired action
function POMDPs.action(p::ToEnd1, b::ParticleCollection{RoombaState})
    n_part = n_particles(b)
    h = ""
    iters = 50
    
    for iter in 1:iters
        j = rand(1:n_part)
        s = particle(b, j)
        last_q = Simulate(b, h, p.d, p.Q, p.N, p.T)
    end
    # Extract the argmax over all actions
    return findmax([last_q[(h,a)] for a in actions(m)])[2]
end

### Simulation and rendering

Here, we will demonstrate how to seed the environment, run a simulation, and render the simulation. To render the simulation, we use the ```Gtk``` package. 

The simulation is carried out using the ```stepthrough``` function defined in the package ```POMDPSimulators.jl```. During a simulation, a window will open that renders the scene. It may be hidden behind other windows on your desktop.

In [41]:
# first seed the environment
Random.seed!(50)

# reset the policy
depth = 1
p = ToEnd1(depth, DefaultDict(0), DefaultDict(0), []) # here, the argument sets the time-steps elapsed to 0

# run the simulation
c = @GtkCanvas()
win = GtkWindow(c, "Roomba Environment", 600, 600)
for (t, step) in enumerate(stepthrough(m, p, belief_updater, max_steps=100))
    @guarded draw(c) do widget
        
        # the following lines render the room, the particles, and the roomba
        ctx = getgc(c)
        set_source_rgb(ctx,1,1,1)
        paint(ctx)
        render(ctx, m, step)
        
        # render some information that can help with debugging
        # here, we render the time-step, the state, and the observation
        move_to(ctx,300,400)
        show_text(ctx, @sprintf("t=%d, state=%s, o=%.3f",t,string(step.s),step.o))
    end
    show(c)
    sleep(0.01) # to slow down the simulation
end

UndefVarError: UndefVarError: last_q not defined

### Specifying initial states and beliefs
If, for debugging purposes, you would like to hard-code a starting location or initial belief state for the roomba, you can do so by taking the following steps.

First, we define the initial state using the following line of code:
```
is = RoombaState(x,y,th,0.)
```
Where ```x``` and ```y``` are the x,y coordinates of the starting location and ```th``` is the heading in radians. The last entry, ```0.```, respresents whether the state is terminal, and should remain unchanged.

If you would like to initialize the Roomba's belief as perfectly localized, you can do so with the following line of code:
```
b0 = Deterministic(is)
```
If you would like to initialize the standard unlocalized belief, use these lines:
```
dist = initialstate_distribution(m)
b0 = initialize_belief(belief_updater, dist)
```
Finally, we call the ```stepthrough``` function using the initial state and belief as follows:
```
stepthrough(m,planner,belief_updater,b0,is,max_steps=300)
```

### Discretizing the state space
Certain POMDP solution techniques require discretizing the state space. Should we need to do so, we first define the state space by specifying the number of points to discretize the range of possible x, y, and $\theta$ values, and then calling the DiscreteRoombaStateSpace constructor.
```
num_x_pts = ... # e.g. 50
num_y_pts = ... # e.g. 50
num_th_pts = ... # e.g. 20
sspace = DiscreteRoombaStateSpace(num_x_pts,num_y_pts,num_th_pts)
```

Next, we pass in the state space as an argument when constructing the POMDP.
```
m = RoombaPOMDP(sensor=sensor, mdp=RoombaMDP(config=config, sspace=sspace))
```

### Discretizing the action space
Certain POMDP solution techniques require discretizing the action space. Should we need to do so, we first define the action space as follows:
```
vlist = [...]
omlist = [...]
aspace = vec(collect(RoombaAct(v, om) for v in vlist, om in omlist))
```
Where ```vlist``` is an array of possible values for the velocity (e.g. ```[0,1,10]```) and ```omlist``` is an array of possible turn-rates (e.g. ```[-1,0,1]```).

Next, we pass in the action space as an argument when constructing the POMDP.
```
m = RoombaPOMDP(sensor=sensor, mdp=RoombaMDP(config=config, aspace=aspace))
```

### Discretizing the Lidar observation space
Certain POMDP solution techniques require discretizing the observation space. The Bumper sensor has a discrete observation space by default, while the Lidar sensor is continuous by default. The observations can take values in the range $[0,\infty)$. Should we need to do discretize the Lidar observation space, we first define cut-points $x_{1:K}$ such that all observations between $-\infty$ and $x_1$ are considered the discrete observation 1, all observations between $x_1$ and $x_2$ are considered discrete observation 2, and so on. All observations between $x_K$ and $\infty$ are considered discrete observation $K+1$. We instantiate the discretized sensor as follows:
```
cut_points = [x_1, x_2, ..., x_K]
sensor = DiscreteLidar(cut_points)
```
Next, we pass in the sensor as an argument when constructing the POMDP as usual.
```
m = RoombaPOMDP(sensor=sensor, mdp=RoombaMDP(config=config))
```






### Evaluation 

Here, we demonstate a simple evaluation of the policy's performance for a few random seeds. This is meant to serve only as an example, and we encourage you to develop your own evaluation metrics.

We intialize the robot using five different random seeds, and simulate its performance for 100 time-steps. We then sum the rewards experienced during its interaction with the environment and track this total reward for the five trials.
Finally, we report the mean and standard error for the total reward. The standard error is the standard deviation of a sample set divided by the square root of the number of samples, and represents the uncertainty in the estimate of the mean value.

In [9]:
using Statistics

total_rewards = []

for exp = 1:5
    println(string(exp))
    
    Random.seed!(exp)
    
    p = ToEnd(depth)
    traj_rewards = sum([step.r for step in stepthrough(m,p,belief_updater, max_steps=100)])
    
    push!(total_rewards, traj_rewards)
end

@printf("Mean Total Reward: %.3f, StdErr Total Reward: %.3f", mean(total_rewards), std(total_rewards)/sqrt(5))

1


UndefVarError: UndefVarError: T not defined